# Architekturkonzepte in LLM-Anwendungen

In diesem Notebook werden wir verschiedene Architekturkonzepte für LLM-Anwendungen untersuchen:

1. Agentenbasierte Ansätze: Grundlagen und Vorteile
2. ReAct Pattern: Reasoning and Acting in LLMs
3. Graph-RAG und Hierarchical RAG: Fortgeschrittene RAG-Architekturen


## Benötigte Bibliotheken installieren

Zunächst müssen wir sicherstellen, dass alle notwendigen Bibliotheken installiert sind:

In [1]:
!pip install langchain langchain-community langchain-openai langgraph python-dotenv

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv

# Laden der Umgebungsvariablen aus der .env-Datei
load_dotenv()

os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY'


# Überprüfen, ob der OpenAI API-Schlüssel gesetzt ist
if os.getenv("OPENAI_API_KEY"):
    print("OpenAI API-Schlüssel ist konfiguriert.")
else:
    print("WARNUNG: OpenAI API-Schlüssel fehlt! Bitte in der .env-Datei konfigurieren.")

OpenAI API-Schlüssel ist konfiguriert.


## 1. Agentenbasierte Ansätze

### Grundlagen von Agenten

Agenten sind KI-Systeme, die selbstständig Entscheidungen treffen und Aktionen ausführen können. Sie kombinieren LLMs mit Tools und einem Entscheidungsprozess, um komplexe Aufgaben zu lösen.

In [3]:
!pip install pytz tzdata

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [14]:
from langchain_openai import OpenAI
from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent, load_tools

# Einfachen LLM initialisieren
llm = OpenAI(temperature=0)

# Einige einfache Tools definieren
tools = load_tools(["llm-math"], llm=llm)

# Eigenes Tool erstellen
@tool
def aktuelle_zeit(timezone: str = "Europe/Berlin") -> str:
    """Gibt die aktuelle Uhrzeit in der angegebenen Zeitzone zurück."""
    from datetime import datetime
    from zoneinfo import ZoneInfo
    try:
        return datetime.now(ZoneInfo(timezone)).strftime("%Y-%m-%d %H:%M:%S %Z")
    except Exception as e:
        return f"Fehler: {str(e)}"

# Tool zur Tool-Liste hinzufügen
tools.append(aktuelle_zeit)

# Einfachen Agenten erstellen
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# Agenten ausführen
agent.invoke("Wie lautet die aktuelle Uhrzeit in Berlin und berechne dann 15 hoch 0.5?")



> Entering new AgentExecutor chain...
 I should first get the current time in Berlin and then use a calculator to calculate 15 to the power of 0.5.
Action: aktuelle_zeit
Action Input: timezone = 'Europe/Berlin'
Observation: Fehler: "No time zone found with key timezone = 'Europe/Berlin'"
Thought: I should try using a different timezone.
Action: aktuelle_zeit
Action Input: timezone = 'Europe/Paris'
Observation: Fehler: "No time zone found with key timezone = 'Europe/Paris'"
Thought: I should try using a different timezone.
Action: aktuelle_zeit
Action Input: timezone = 'Europe/London'
Observation: Fehler: "No time zone found with key timezone = 'Europe/London'"
Thought: I should try using a different timezone.
Action: aktuelle_zeit
Action Input: timezone = 'Europe/Moscow'
Observation: Fehler: "No time zone found with key timezone = 'Europe/Moscow'"
Thought: I should try using a different timezone.
Action: aktuelle_zeit
Action Input: timezone = 'Europe/Rome'
Observation: Fehler: "No ti

{'input': 'Wie lautet die aktuelle Uhrzeit in Berlin und berechne dann 15 hoch 0.5?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

### Vorteile agentenbasierter Ansätze

- **Autonomie**: Agenten können eigenständig Entscheidungen treffen und Aktionen ausführen
- **Flexibilität**: Können mit vielen verschiedenen Tools arbeiten
- **Komplexität**: Können mehrstufige Probleme in Teilschritte zerlegen
- **Erweiterbarkeit**: Leichte Integration neuer Tools und Funktionen

![Agent Architektur](https://upload.wikimedia.org/wikipedia/commons/2/21/Agent_based_modelling.png)

## 2. ReAct Pattern: Reasoning and Acting in LLMs

Das ReAct-Muster kombiniert Reasoning (Denken) und Acting (Handeln) in einem iterativen Prozess.

### Komponenten des ReAct-Musters:

1. **Thought (Gedanke)**: Das LLM überlegt, wie es vorgehen soll
2. **Action (Aktion)**: Das LLM wählt eine Aktion/Tool aus und führt sie aus
3. **Observation (Beobachtung)**: Das LLM erhält die Ergebnisse der Aktion
4. **Wiederholen**: Bis die Aufgabe gelöst ist

In [15]:
# Implementierung des ReAct-Musters mit LangChain
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_openai import OpenAI

# LLM initialisieren
llm = OpenAI(temperature=0)

# Tools laden (wir nutzen serpapi für Web-Suche und llm-math für Berechnungen)
# Hinweis: Sie benötigen einen SERPAPI-Schlüssel in der .env-Datei
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# ReAct-Agent initialisieren
react_agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Dies ist der ReAct-Agent
    verbose=True  # Ausführliche Ausgabe, um den Denkprozess zu sehen
)

# Komplexe Frage stellen, die mehrere Schritte erfordert
react_agent.invoke(
    "Wie alt ist Angela Merkel und was ist diese Zahl quadriert?"
)



> Entering new AgentExecutor chain...
 I should use the search engine to find information about Angela Merkel's age.
Action: Search
Action Input: "Angela Merkel age"
Observation: 70 years
Thought: Now I can use the calculator to square this number.
Action: Calculator
Action Input: 70
Observation: Answer: 70
Thought: I now know the final answer.
Final Answer: 4900

> Finished chain.


{'input': 'Wie alt ist Angela Merkel und was ist diese Zahl quadriert?',
 'output': '4900'}

### Funktionsweise des ReAct-Musters

1. Das LLM wird mit einer **promt template** angewiesen, seine Gedanken zu verbalisieren
2. Der Agent formuliert einen **Gedanken** darüber, wie er die Aufgabe angehen sollte
3. Der Agent wählt eine **Aktion** und ein Tool aus der verfügbaren Tool-Liste
4. Das Tool wird ausgeführt und liefert eine **Beobachtung** zurück
5. Der Agent überlegt, basierend auf der Beobachtung, was als nächstes zu tun ist
6. Dieser Prozess wiederholt sich, bis der Agent glaubt, die Aufgabe gelöst zu haben

### Vorteile des ReAct-Musters:

- **Transparenz**: Der Denkprozess des Agenten ist sichtbar
- **Bessere Entscheidungen**: Durch explizites Nachdenken werden bessere Entscheidungen getroffen
- **Selbstkorrektur**: Der Agent kann Fehler erkennen und korrigieren
- **Systematisches Vorgehen**: Komplexe Probleme werden strukturiert gelöst

## 3. Graph-RAG und Hierarchical RAG

RAG (Retrieval Augmented Generation) verbessert LLM-Antworten durch die Einbindung externer Informationen. Graph-RAG und Hierarchical RAG sind fortgeschrittene Architekturen, die die Leistung von RAG-Systemen verbessern.

### Graph-RAG

Graph-RAG nutzt Graphstrukturen, um Beziehungen zwischen Dokumenten und Konzepten zu modellieren.

**Schlüsselkonzepte:**
- Dokumente und Informationen werden als Knoten in einem Graphen dargestellt
- Beziehungen zwischen Informationen werden als Kanten modelliert
- Der Graph ermöglicht kontextreicheres Retrieval

In [16]:
# Einfaches Beispiel für Graph-RAG mit LangGraph
from typing import Annotated, TypedDict, List, Dict, Any
from langgraph.graph import StateGraph
import operator
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser

# Definieren eines einfachen Zustands für unseren Graphen
class GraphState(TypedDict):
    query: str
    context: List[str]
    answer: str

# Embeddings initialisieren
embeddings = OpenAIEmbeddings()

# LLM initialisieren
llm = ChatOpenAI(temperature=0)

# Beispieldaten für die Vektordatenbank
sample_texts = [
    "Berlin ist die Hauptstadt von Deutschland.",
    "München ist die Hauptstadt von Bayern.",
    "Hamburg ist die zweitgrößte Stadt Deutschlands.",
    "Frankfurt ist ein wichtiges Finanzzentrum in Europa.",
    "Köln ist bekannt für seinen Dom."
]

# Vektordatenbank erstellen
vectorstore = Chroma.from_texts(sample_texts, embeddings)

# Knoten-Funktionen definieren
def retrieve(state: GraphState) -> GraphState:
    """Dokumente aus der Vektordatenbank abrufen"""
    query = state["query"]
    # Top 2 relevante Dokumente abrufen
    docs = vectorstore.similarity_search(query, k=2)
    return {"context": [doc.page_content for doc in docs]}

def generate_answer(state: GraphState) -> GraphState:
    """Antwort basierend auf dem Kontext generieren"""
    query = state["query"]
    context = state["context"]
    
    prompt = ChatPromptTemplate.from_template(
        """Du bist ein hilfreicher Assistent. 
        Verwende den folgenden Kontext, um die Frage zu beantworten.
        
        Kontext: {context}
        
        Frage: {query}
        """
    )
    
    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"context": "\n".join(context), "query": query})
    
    return {"answer": answer}

# Graph erstellen
graph = StateGraph(GraphState)

# Knoten hinzufügen
graph.add_node("retrieve", retrieve)
graph.add_node("generate", generate_answer)

# Kanten definieren
graph.set_entry_point("retrieve")
graph.add_edge("retrieve", "generate")
graph.set_finish_point("generate")

# Graph kompilieren
chain = graph.compile()

# Graph testen
result = chain.invoke({"query": "Was ist die Hauptstadt von Deutschland?", "context": [], "answer": ""})
print("Antwort:", result["answer"])

Antwort: Die Hauptstadt von Deutschland ist Berlin.


### Hierarchical RAG

Hierarchical RAG organisiert Informationen in Hierarchien, um den Suchraum effizient einzugrenzen.

**Schlüsselkonzepte:**
- Informationen werden in verschiedenen Ebenen organisiert
- Die Suche beginnt auf einer hohen Ebene und verfeinert sich schrittweise
- Ermöglicht effizientere Informationsextraktion bei großen Dokumentenmengen

#### Beispielarchitektur für Hierarchical RAG:

1. **Oberste Ebene**: Dokumenttitel und Zusammenfassungen
2. **Mittlere Ebene**: Abschnitte und Kapitel
3. **Unterste Ebene**: Detaillierte Textpassagen

In [17]:
# Konzeptionelles Beispiel für Hierarchical RAG (vereinfacht)
from typing import List, Dict
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# LLM initialisieren
llm = ChatOpenAI(temperature=0)

# Vereinfachte Dokument-Hierarchie
document_hierarchy = {
    "level1": [
        {"id": "doc1", "summary": "Deutschlands Großstädte und ihre Bedeutung"},
        {"id": "doc2", "summary": "Europäische Hauptstädte im Vergleich"}
    ],
    "level2": {
        "doc1": [
            {"id": "doc1_section1", "title": "Berlin als Hauptstadt"},
            {"id": "doc1_section2", "title": "Hamburg als Handelsmetropole"}
        ],
        "doc2": [
            {"id": "doc2_section1", "title": "Berlin im europäischen Kontext"},
            {"id": "doc2_section2", "title": "Paris als Kulturzentrum"}
        ]
    },
    "level3": {
        "doc1_section1": "Berlin ist die Hauptstadt und bevölkerungsreichste Stadt Deutschlands. Mit rund 3,7 Millionen Einwohnern ist Berlin auch die größte Stadt der Europäischen Union.",
        "doc1_section2": "Hamburg ist mit 1,8 Millionen Einwohnern die zweitgrößte Stadt Deutschlands und ein bedeutendes Wirtschafts- und Handelszentrum in Nordeuropa.",
        "doc2_section1": "Im Vergleich zu anderen europäischen Hauptstädten hat Berlin eine besondere Geschichte aufgrund der deutschen Teilung im 20. Jahrhundert.",
        "doc2_section2": "Paris, die Hauptstadt Frankreichs, gilt als eines der wichtigsten Kulturzentren Europas mit berühmten Museen wie dem Louvre und dem Centre Pompidou."
    }
}

def hierarchical_search(query: str) -> str:
    """Führt eine hierarchische Suche durch"""
    
    # 1. Schritt: Auswahl relevanter Dokumente auf Level 1
    level1_prompt = ChatPromptTemplate.from_template(
        """Gegeben sind die folgenden Dokumentzusammenfassungen:
        {summaries}
        
        Für die Anfrage: {query}
        Gib die ID des relevantesten Dokuments zurück. Antworte nur mit der ID."""
    )
    
    # Zusammenfassungen zusammenstellen
    summaries = "\n".join([f"ID: {doc['id']}, Zusammenfassung: {doc['summary']}" 
                           for doc in document_hierarchy["level1"]])
    
    # Level 1 Auswahl treffen
    level1_chain = level1_prompt | llm | StrOutputParser()
    selected_doc = level1_chain.invoke({"summaries": summaries, "query": query})
    print(f"Ausgewähltes Dokument Level 1: {selected_doc}")
    
    # 2. Schritt: Auswahl relevanter Abschnitte auf Level 2
    level2_prompt = ChatPromptTemplate.from_template(
        """Gegeben sind die folgenden Abschnitte aus dem Dokument {doc_id}:
        {sections}
        
        Für die Anfrage: {query}
        Gib die ID des relevantesten Abschnitts zurück. Antworte nur mit der ID."""
    )
    
    # Abschnitte zusammenstellen
    sections = "\n".join([f"ID: {section['id']}, Titel: {section['title']}" 
                          for section in document_hierarchy["level2"].get(selected_doc.strip(), [])])
    
    # Level 2 Auswahl treffen
    level2_chain = level2_prompt | llm | StrOutputParser()
    selected_section = level2_chain.invoke({"doc_id": selected_doc, "sections": sections, "query": query})
    print(f"Ausgewählter Abschnitt Level 2: {selected_section}")
    
    # 3. Schritt: Detailtext abrufen und Antwort generieren
    detail_text = document_hierarchy["level3"].get(selected_section.strip(), "Keine Informationen gefunden.")
    
    answer_prompt = ChatPromptTemplate.from_template(
        """Basierend auf dem folgenden Text, beantworte die Frage.
        
        Text: {text}
        
        Frage: {query}
        
        Antwort:"""
    )
    
    answer_chain = answer_prompt | llm | StrOutputParser()
    answer = answer_chain.invoke({"text": detail_text, "query": query})
    
    return answer

# Testen der hierarchischen Suche
result = hierarchical_search("Wie viele Einwohner hat Berlin?")
print("\nAntwort:", result)

Ausgewähltes Dokument Level 1: doc1
Ausgewählter Abschnitt Level 2: doc1_section1

Antwort: Berlin hat rund 3,7 Millionen Einwohner.


## Zusammenfassung

In diesem Notebook haben wir drei wichtige Architekturkonzepte für LLM-Anwendungen kennengelernt:

1. **Agentenbasierte Ansätze**:
   - Agenten kombinieren LLMs mit Tools für komplexe Aufgaben
   - Sie bieten Autonomie, Flexibilität und einfache Erweiterbarkeit

2. **ReAct Pattern**:
   - Kombiniert Reasoning und Acting in einem iterativen Prozess
   - Verbessert die Transparenz und Qualität von Entscheidungen
   - Ermöglicht systematisches Problemlösen

3. **Graph-RAG und Hierarchical RAG**:
   - Graph-RAG nutzt Beziehungen zwischen Informationen
   - Hierarchical RAG organisiert Wissen in verschiedenen Abstraktionsebenen
   - Beide verbessern die Effektivität von RAG-Systemen

Diese Konzepte können kombiniert werden, um leistungsfähige KI-Systeme zu entwickeln, die komplexe Aufgaben effizient lösen können.

## Übungen

1. Erweitern Sie den ReAct-Agenten um ein eigenes Tool (z.B. Wetter-API oder Währungsumrechnung)
2. Modifizieren Sie den Graph-RAG-Ansatz, um einen Feedback-Knoten einzubauen, der die Antwort verbessert
3. Entwerfen Sie eine hierarchische RAG-Struktur für ein spezielles Fachgebiet (z.B. Medizin oder Rechtswesen)